yfinance - FINANCIAL RATIOS

In [6]:
import yfinance as yf

def get_financials(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        "PE Ratio": info.get('trailingPE'),
        "EPS": info.get('trailingEps'),
        "DE Ratio": info.get('debtToEquity'),
        "ROE": info.get('returnOnEquity'),
    }

# Example usage:
tickerSymbol = "AAPL"
financials = get_financials(tickerSymbol)
for key, value in financials.items():
    print(f"{key}: {value}")

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [4]:
yf.Ticker("AAPL").info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '408 996 1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscriptio

In [5]:

# Define the ticker symbol
tickerSymbol = 'AAPL'

# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
tickerDf = tickerData.history(period='1d', start='2010-1-1', end='2020-12-31')

# See your data
print(tickerDf)

                                 Open        High         Low       Close  \
Date                                                                        
2010-01-04 00:00:00-05:00    6.478689    6.511169    6.446817    6.496295   
2010-01-05 00:00:00-05:00    6.514204    6.544255    6.473224    6.507525   
2010-01-06 00:00:00-05:00    6.507526    6.533328    6.397337    6.404015   
2010-01-07 00:00:00-05:00    6.427691    6.435280    6.345732    6.392175   
2010-01-08 00:00:00-05:00    6.383678    6.435282    6.346039    6.434675   
...                               ...         ...         ...         ...   
2020-12-23 00:00:00-05:00  130.213127  130.479139  128.853451  129.030807   
2020-12-24 00:00:00-05:00  129.385521  131.493996  129.168761  130.025940   
2020-12-28 00:00:00-05:00  132.016154  135.316795  131.543214  134.676376   
2020-12-29 00:00:00-05:00  136.016388  136.745477  132.361033  132.883224   
2020-12-30 00:00:00-05:00  133.582753  133.986717  131.434859  131.750153   

WEB SCRAPPING YAHOO

In [5]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

symbol = 'AAPL'

url = 'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol

# Set up the request headers that we're going to use, to simulate
# a request by the Chrome browser. Simulating a request from a browser
# is generally good practice when building a scraper
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'close',
    'DNT': '1', # Do Not Track Request Header 
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

# Fetch the page that we're going to parse, using the request headers
# defined above
page = requests.get(url, headers=headers)

# Parse the page with LXML, so that we can start doing some XPATH queries
# to extract the data that we want
tree = html.fromstring(page.content)

# Smoke test that we fetched the page by fetching and displaying the H1 element
tree.xpath("//h1/text()")

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [18]:
table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

# Ensure that some table rows are found; if none are found, then it's possible
# that Yahoo Finance has changed their page layout, or have detected
# that you're scraping the page.
assert len(table_rows) > 0

parsed_rows = []

for table_row in table_rows:
    parsed_row = []
    el = table_row.xpath("./div")
    
    none_count = 0
    
    for rs in el:
        try:
            (text,) = rs.xpath('.//span/text()[1]')
            parsed_row.append(text)
        except ValueError:
            parsed_row.append(np.NaN)
            none_count += 1

    if (none_count < 4):
        parsed_rows.append(parsed_row)

df = pd.DataFrame(parsed_rows)
df

,0,1,2,3,4
0,Breakdown,9/30/2022,9/30/2021,9/30/2020,9/30/2019
1,Total Assets,"352,755,000","351,002,000","323,888,000","338,516,000"
2,Total Liabilities Net Minority Interest,"302,083,000","287,912,000","258,549,000","248,028,000"
3,Total Equity Gross Minority Interest,"50,672,000","63,090,000","65,339,000","90,488,000"
4,Total Capitalization,"149,631,000","172,196,000","164,006,000","182,295,000"
5,Common Stock Equity,"50,672,000","63,090,000","65,339,000","90,488,000"
6,Net Tangible Assets,"50,672,000","63,090,000","65,339,000","90,488,000"
7,Working Capital,"-18,577,000","9,355,000","38,321,000","57,101,000"
8,Invested Capital,"170,741,000","187,809,000","177,775,000","198,535,000"
9,Tangible Book Value,"50,672,000","63,090,000","65,339,000","90,488,000"


In [20]:
# NOT FINISH YET, TO CONTINUE FOLLOW LINK
linko = "https://www.mattbutton.com/2019/01/24/how-to-scrape-yahoo-finance-and-extract-fundamental-stock-market-data-using-python-lxml-and-pandas/#:~:text=In%20this%20blog%20post%20I%E2%80%99ll%20show%20you%20how,work%20for%20any%20stock%20symbol%20on%20Yahoo%20Finance."

BEST LIBRARY

In [1]:
import fundamentalanalysis as fa

ticker = "AAPL"
api_key = "56bd4a3ac58b13bd6c93c37ff7f06dac"

In [3]:
# Show Key Metrics
key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
key_metrics_annually

,2022,2021,2020,2019,2018
period,FY,FY,FY,FY,FY
revenuePerShare,24.317273,21.903541,15.820258,14.085283,13.399334
netIncomePerShare,6.154614,5.669029,3.308587,2.991446,3.003354
operatingCashFlowPerShare,7.532763,6.229346,4.64923,3.756685,3.906565
freeCashFlowPerShare,6.872426,5.565624,4.228014,3.188508,3.23492
cashPerShare,2.978793,3.750553,5.241031,5.443948,3.344902
bookValuePerShare,3.124822,3.777557,3.765477,4.898834,5.405593
tangibleBookValuePerShare,3.124822,3.777557,3.765477,4.898834,5.405593
shareholdersEquityPerShare,3.124822,3.777557,3.765477,4.898834,5.405593
interestDebtPerShare,7.585118,7.626006,6.64524,6.043039,5.939155


In [9]:
import numpy as np
df = np.transpose(key_metrics_annually)

df['stock'] = ticker

df

,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,...,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare,stock
2022,FY,24.317273,6.154614,7.532763,6.872426,2.978793,3.124822,3.124822,3.124822,7.585118,...,5763000000,56.400205,104.685277,8.075698,6.471608,3.486641,45.197331,1.969589,-0.660337,AAPL
2021,FY,21.903541,5.669029,6.229346,5.565624,3.750553,3.777557,3.777557,3.777557,7.626006,...,5320500000,51.390969,93.851071,11.276593,7.102415,3.88914,32.367933,1.500713,-0.663722,AAPL
2020,FY,15.820258,3.308587,4.64923,4.228014,5.241031,3.765477,3.765477,3.765477,6.64524,...,4083500000,49.787534,91.04819,8.741883,7.331152,4.008866,41.753016,0.878664,-0.421217,AAPL
2019,FY,14.085283,2.991446,3.756685,3.188508,5.443948,4.898834,4.898834,4.898834,6.043039,...,4031000000,64.258765,104.314077,9.263639,5.680159,3.499048,39.401364,0.610645,-0.568178,AAPL
2018,FY,13.399334,3.003354,3.906565,3.23492,3.344902,5.405593,5.405593,5.405593,5.939155,...,4405500000,67.332499,124.570214,8.817631,5.420859,2.930074,41.394338,0.555601,-0.671644,AAPL


In [21]:
# Show a large set of in-depth ratios
financial_ratios_annually = fa.financial_ratios(ticker, api_key, period="annual")
financial_ratios_annually

,2022,2021,2020,2019,2018
period,FY,FY,FY,FY,FY
currentRatio,0.879356,1.074553,1.363604,1.540126,1.123843
quickRatio,0.709408,0.90966,1.218195,1.384447,0.986566
cashRatio,0.153563,0.278449,0.36071,0.462022,0.221733
daysOfSalesOutstanding,56.400205,51.390969,49.787534,64.258765,67.332499
daysOfInventoryOutstanding,8.075698,11.276593,8.741883,9.263639,8.817631
operatingCycle,64.475903,62.667561,58.529418,73.522404,76.15013
daysOfPayablesOutstanding,104.685277,93.851071,91.04819,104.314077,124.570214
cashConversionCycle,-40.209374,-31.18351,-32.518772,-30.791673,-48.420084
grossProfitMargin,0.433096,0.417794,0.382332,0.378178,0.383437


In [22]:
fa.financial_statement_growth(ticker, api_key, period="annual")

,2022,2021,2020,2019,2018
period,FY,FY,FY,FY,FY
revenueGrowth,0.077938,0.332594,0.055121,-0.020411,0.15862
grossProfitGrowth,0.11742,0.456191,0.066713,-0.033848,0.15482
ebitgrowth,0.096265,0.64357,0.036884,-0.098282,0.155745
operatingIncomeGrowth,0.096265,0.64357,0.036884,-0.098282,0.155745
netIncomeGrowth,0.054109,0.649161,0.039,-0.071811,0.231226
epsgrowth,0.084656,0.712991,0.107023,-0.003333,0.293103
epsdilutedGrowth,0.089127,0.710366,0.104377,-0.003356,0.295652
weightedAverageSharesGrowth,-0.029058,-0.037508,-0.060592,-0.068117,-0.050192
weightedAverageSharesDilutedGrowth,-0.031966,-0.037842,-0.057403,-0.070238,-0.047905
